In [207]:
import numpy as np
from scipy.optimize import fsolve

# Rocket engine thrust as described in 
# https://www.grc.nasa.gov/www/k-12/airplane/rktthsum.html


p_t = 300. * 10**5  # pressure in tank [Pa]
T_t = 300.          # temperature in tank [K]
d_star = .04        # diameter at throat [m]
d_e = .6            # diameter at exit [m]

gam = 1.4           # specific heat ratio for oxygen
R = 287.058         # specific gas constant for oxygen [J/kg/K]
p_0 = 862.          # vacuum tube pressure [Pa]


A_e = np.pi * (d_e/2.)**2.
A_star = np.pi * (d_star/2.)**2.


m_dot = (A_star * p_t / np.sqrt(T_t)) * np.sqrt(gam/R) * ((gam+1.)/2.)**(-((gam+1.)/(gam-1.)/2.))
M_e = fsolve(
    lambda M: (((gam+1.)/2.)**(-((gam+1.)/(gam-1.)/2))) / M * (1. + M**2. * (gam-1.)/2.)**((gam+1.)/(gam-1.)/2.) - A_e/A_star
    , 100.)[0]
p_e = p_t * (1. + M_e**2. * (gam-1.)/2.)**(-(gam/(gam-1.)))
T_e = T_t * (1. + M_e**2. * (gam-1.)/2.)**(-1.)
v_e = M_e * np.sqrt(gam * R * T_e)
F = m_dot * v_e + (p_e - p_0) * A_e

print """
Thrust: {} [kN]
m_dot:  {} [kg/s]
v_e:    {} [m/s]
M_e:    {} 
p_e:    {} [Pa]
T_e:    {} [K]
""".format(F/1000., m_dot, v_e, M_e, p_e, T_e)

print """
m_dot * v_e:       {} [kN]
(p_e - p_0) * A_e: {} [kN]
""".format(m_dot * v_e/1000., (p_e - p_0) * A_e/1000.)


Thrust: 4.1162089094 [kN]
m_dot:  5.49776232808 [kg/s]
v_e:    738.966762721 [m/s]
M_e:    6.93629038982 
p_e:    7679.59674248 [Pa]
T_e:    28.242139017 [K]


m_dot * v_e:       4.06266362979 [kN]
(p_e - p_0) * A_e: 0.0535452796033 [kN]



In [175]:
(m_dot*v_e, (p_e-p_0)*A_e)

(65945.482393242914, 440.05996174336082)

In [151]:
(1. + 10.**2. * (gam-1.)/2.)**(-(gam/(gam-1.)))

2.3563102282258094e-05

In [171]:
A_e, A_star

(0.2827433388230814, 0.0012566370614359172)